# sentiment Analysis

In [1]:
import pandas as pd 
# Read the JSON file into a DataFrame
df = pd.read_json("C:\\Users\\DELL\\Desktop\\CS-652 Text Based Analysis\\Datasets\\kindle_reviews.json",lines=True)
# Display a random sample of 10 rows from the DataFrame
df.sample(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
759859,A28AIE80JKI8RR,B00HCLZHXA,Christine Pauls,"[0, 0]","What can I say? Five magnificent authors, thre...",5,Fabulous!,1388275200,"12 29, 2013"
109299,A1P9KJ0BOZPWWY,B005M3KMZS,SewAmused,"[8, 10]",We have an English (as from England) author wr...,2,OH PLEASE!,1338595200,"06 2, 2012"
524768,A2CZKERY4LUBVI,B00CSCE9PU,Amy,"[0, 0]",This is a really great series that contains a ...,5,Great series.,1401580800,"06 1, 2014"
731279,A18O9A44NSD4K7,B00GU7XWKC,J. Duarte,"[0, 0]","I read a lot of books, they often cause me to ...",5,Wonderful story,1386547200,"12 9, 2013"
630327,A3LJST1RMWS5X0,B00ETN5GXK,"Mustang Fyant ""The Laughing Cowgirl""","[0, 0]","No matter how a generation may find you, your...",5,Surprises always find you.,1391126400,"01 31, 2014"
775751,AY9BM67KFIQ5E,B00HLTIN9I,Mamawiseman,"[1, 1]",Who could blame her!!! Happy Birthday Samantha...,5,Must Read!!!!,1390521600,"01 24, 2014"
878906,AD5PBM35ISJBZ,B00JBUFFNC,Candy Lyn,"[1, 1]","I seriously enjoyed this story. Yeah, it was s...",5,Funny erotic sensual sexy frustrating,1401580800,"06 1, 2014"
909837,AEAFST0PI00MZ,B00JXW6GFE,T's Book,"[4, 6]","Anyone who has read The Burg Series, especiall...",4,It was like a Burg family reunion! LOVE!,1404777600,"07 8, 2014"
394657,A323H7CL7G313K,B00AUBKOX2,S L Thomas,"[0, 0]",This is not your typical romance story. I love...,5,Great story,1377907200,"08 31, 2013"
274352,A2QMYRCTLV7XN5,B008SEHGFC,Cindy,"[0, 0]","The story line was great, but there are so man...",2,confusing,1377648000,"08 28, 2013"


In [2]:
# Import necessary libraries
from sklearn import preprocessing
import nltk
# Download the opinion lexicon from NLTK
nltk.download('opinion_lexicon')
# Import the opinion lexicon from NLTK corpus
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
# Print total number of words in opinion lexicon
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
# Print examples of positive words in opinion lexicon
print('Examples of positive words in opinion lexicon',
      opinion_lexicon.positive()[:10])
# Print examples of negative words in opinion lexicon
print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])


Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [3]:
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}
 
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
        word_dict[word] = pos_score
      
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
        word_dict[word] = neg_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def bing_liu_score(text):
     # Initialize sentiment score
    sentiment_score = 0
     # Tokenize text into words
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:# Check if word exists in the sentiment dictionary
            sentiment_score += word_dict[word] # Add sentiment score of the word
    return sentiment_score  # Return the total sentiment score

In [5]:
# Fill missing values in the 'text' column of DataFrame 'df' with 'no review'
df['text'].fillna('no review', inplace=True)

In [6]:
# Apply the bing_liu_score function to each row in the 'text' column of DataFrame 'df' and store the results in a new column 'Bing_Liu_Score'
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

In [7]:
# Display the first 10 rows of columns 'overall', 'text', and 'Bing_Liu_Score' from DataFrame 'df'
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,5,I enjoy vintage books and movies so I enjoyed ...,-1
1,4,This book is a reissue of an old one; the auth...,6
2,4,This was a fairly interesting read. It had ol...,7
3,5,I'd never read any of the Amy Brewster mysteri...,0
4,4,"If you like period pieces - clothing, lingo, y...",1
5,4,A beautiful in-depth character description mak...,2
6,4,I enjoyed this one tho I'm not sure why it's c...,3
7,4,Never heard of Amy Brewster. But I don't need ...,1
8,5,Darth Maul working under cloak of darkness com...,0
9,4,This is a short story focused on Darth Maul's ...,3


In [8]:
# Group the DataFrame 'df' by the 'overall' column and calculate the mean of the 'Bing_Liu_Score' column for each group
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.537970
2,0.450044
3,2.127835
4,3.542563
5,3.579195
